<a href="https://colab.research.google.com/github/LGBFBenchmarking/ChangeLogMaker/blob/main/Automate_differences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Test code for creating LGBF change log for Monthly refreshes. The below code find the difference between 2 dataframes/datasets.

#Load files

##new_df is new data including new year's data
##diff_df has old data (no changes or new data for the new year)

#Loads the necessary packages for the code. Important to run this in the begining of the session.

In [1]:
import pandas as pd
import numpy as np

def get_data(url): # this function gets data stored as csv file from a given url
  df = pd.read_csv(url)
  return df


Upload the data sets for the current month and previous month. Copy the url for the raw data files for the two months and paste in respective lines here- current month is new and previous month is old. For ex. in May refresh April data is old_df and May data is new_df.

In [2]:
url_old = "https://raw.githubusercontent.com/LGBFBenchmarking/ChangeLogMaker/refs/heads/main/LGBF_Data_June25.csv"
url_new = "https://raw.githubusercontent.com/LGBFBenchmarking/ChangeLogMaker/refs/heads/main/LGBF_Data_July25.csv"
old_df = get_data(url_old)
new_df = get_data(url_new)

Below checks the column names - make sure they match.

In [3]:
old_df.dtypes

,0
Code,object
LocalAuthority,object
Year,object
Indicator,float64
Numerator,float64
Denominator,float64


In [4]:
new_df.dtypes

,0
Code,object
LocalAuthority,object
Year,object
Indicator,float64
Numerator,float64
Denominator,float64


This sets the formatting or we will have look through way more lines in the differences. So important to run these.

In [5]:
new_df['Indicator'] = new_df['Indicator'].map("{:.2f}".format)
new_df['Numerator'] = new_df['Numerator'].map('{:.2f}'.format)
new_df['Denominator'] = new_df['Denominator'].map('{:.2f}'.format)

In [6]:
old_df['Indicator'] = old_df['Indicator'].map('{:.2f}'.format)
old_df['Numerator'] = old_df['Numerator'].map('{:.2f}'.format)
old_df['Denominator'] = old_df['Denominator'].map('{:.2f}'.format)

In [7]:
old_df = old_df.round(4)
new_df = new_df.round(4)

In [8]:
old_df.head()

,Code,LocalAuthority,Year,Indicator,Numerator,Denominator
0,C&L01,Aberdeen City,2010-11,0.33,637.00,1922292.00
1,C&L01,Aberdeen City,2011-12,0.76,1554.00,2045051.00
2,C&L01,Aberdeen City,2012-13,3.64,7883.00,2163756.00
3,C&L01,Aberdeen City,2013-14,3.48,7726.00,2222588.00
4,C&L01,Aberdeen City,2014-15,3.02,7517.00,2487138.00


In [9]:
new_df.head()

,Code,LocalAuthority,Year,Indicator,Numerator,Denominator
0,C&L01,Aberdeen City,2010-11,0.33,637.00,1922292.00
1,C&L01,Aberdeen City,2011-12,0.76,1554.00,2045051.00
2,C&L01,Aberdeen City,2012-13,3.64,7883.00,2163756.00
3,C&L01,Aberdeen City,2013-14,3.48,7726.00,2222588.00
4,C&L01,Aberdeen City,2014-15,3.02,7517.00,2487138.00


The below lines is to make sure that the original files are not messed with.

In [10]:
df1 = old_df
df2 = new_df

This finds the new data for this month.

In [11]:
newdf = df2[~df2.astype(str).apply(tuple, 1).isin(df1.astype(str).apply(tuple, 1))]

In [12]:
newdf

,Code,LocalAuthority,Year,Indicator,Numerator,Denominator
293,C&L01,North Ayrshire,2023-24,8.40,6122.00,728937.00
1049,C&L03,East Renfrewshire,2023-24,nan,nan,0.00
1159,C&L03,Midlothian,2021-22,nan,30.00,0.00
1160,C&L03,Midlothian,2022-23,nan,19.00,0.00
1161,C&L03,Midlothian,2023-24,nan,7.00,0.00
...,...,...,...,...,...,...
39313,SW06,Scotland,2020-21,120.27,91993.00,764877.00
39314,SW06,Scotland,2021-22,106.85,98277.00,919768.00
39315,SW06,Scotland,2022-23,102.31,97261.00,950638.00
39316,SW06,Scotland,2023-24,104.43,104963.00,1005096.00


the below code downloads the new data into your computer as an excel file

In [13]:
from google.colab import files
newdf. to_excel('newdf.xlsx')
files.download('newdf.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The below code is a function to find differences. This needs to be run before the line below.

In [14]:
def dataframe_difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(
        df2,
        indicator=True,
        how='outer'
    )
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']  # Identify data changes/updates
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    return diff_df

Another functions to define the column names when we merge. So run this code before the next set of lines.

In [15]:
def getColChangedName(row):
    colsChanged = []
    for c in df1.columns.values:
        if row[c+"_diff"] == True:
            colsChanged.append(c)
    return ", ".join(colsChanged)

Same as above when we found the new data. This one is reassigning the data sets so the original data is not messed with, then merges the two data sets for comparing.

In [16]:
df1 = old_df.set_index(['Code', 'LocalAuthority', 'Year'])
df2 = new_df.set_index(['Code','LocalAuthority', 'Year'])

diffs = df1.merge(df2, left_index=True, right_index=True, suffixes=('_old', '_new'))

diff_cols = [c+"_diff" for c in df1.columns.values]

for c in df1.columns.values:
    diffs[c+"_diff"] = diffs[c+"_old"] != diffs[c+"_new"]

diffs["Record_Changed"] = diffs[diff_cols].sum(axis=1)/len(diff_cols) > 0


diffs = diffs[diffs["Record_Changed"] == 1]
diffs["Cols_Changed"] = diffs.apply(lambda row: getColChangedName(row), axis=1)

diffs

Indicator_old Numerator_old Denominator_old  \
Code  LocalAuthority    Year                                                  
C&L01 North Ayrshire    2023-24          8.35       6122.00       732937.00   
C&L03 East Renfrewshire 2023-24           nan           nan             nan   
      Midlothian        2021-22           nan         30.00             nan   
                        2022-23           nan         19.00             nan   
                        2023-24           nan          7.00             nan   
...                                       ...           ...             ...   
SW06  Scotland          2019-20        104.89     112873.00      1076118.00   
                        2020-21        120.20      91887.00       764451.00   
                        2021-22        106.74      98102.00       919043.00   
                        2022-23        102.22      97128.00       950151.00   
                        2023-24        103.95     102941.00       990306.00   

                                Indicator_new Numerator_new Denominator_new  \
Code  LocalAuthority    Year                                                  
C&L01 North Ayrshire    2023-24          8.40       6122.00       728937.00   
C&L03 East Renfrewshire 2023-24           nan           nan            0.00   
      Midlothian        2021-22           nan         30.00            0.00   
                        2022-23           nan         19.00            0.00   
                        2023-24           nan          7.00            0.00   
...                                       ...           ...             ...   
SW06  Scotland          2019-20        104.88     112894.00      1076404.00   
                        2020-21        120.27      91993.00       764877.00   
                        2021-22        106.85      98277.00       919768.00   
                        2022-23        102.31      97261.00       950638.00   
                        2023-24        104.43     104963.00      1005096.00   

                                 Indicator_diff  Numerator_diff  \
Code  LocalAuthority    Year                                      
C&L01 North Ayrshire    2023-24            True           False   
C&L03 East Renfrewshire 2023-24           False           False   
      Midlothian        2021-22           False           False   
                        2022-23           False           False   
                        2023-24           False           False   
...                                         ...             ...   
SW06  Scotland          2019-20            True            True   
                        2020-21            True            True   
                        2021-22            True            True   
                        2022-23            True            True   
                        2023-24            True            True   

                                 Denominator_diff  Record_Changed  \
Code  LocalAuthority    Year                                        
C&L01 North Ayrshire    2023-24              True            True   
C&L03 East Renfrewshire 2023-24              True            True   
      Midlothian        2021-22              True            True   
                        2022-23              True            True   
                        2023-24              True            True   
...                                           ...             ...   
SW06  Scotland          2019-20              True            True   
                        2020-21              True            True   
                        2021-22              True            True   
                        2022-23              True            True   
                        2023-24              True            True   

                                                      Cols_Changed  
Code  LocalAuthority    Year                                        
C&L01 North Ayrshire    2023-24             Indicator, Denominator  

This downloads the differences between the two data sets there by finding any updates to previously published data. Unfortunately, due to formatting changes/rounding and sometimes lack of any data without NAs, we may see lines of data showing no changes. Simply check them and delete manually. Unfortunately, this code does pick up no changes due to strict checking. Better this than missing any updates.

In [17]:
from google.colab import files
diffs. to_excel('diffs.xlsx')
files.download('diffs.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Use diff and newdf to get the Change log